In [94]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate as sci
import time
from getdist import plots, MCSamples
import time

import tensorflow as tf
ds = tf.contrib.distributions

%matplotlib inline
dpi = 200

In [110]:
def likelihood(mod, dat, sigma): # retorna escalar, log(L)
    sig = tf.cast(tf.diag_part(sigma), tf.float32) #np.diagonal(sigma)
    L = -0.5*chi2(mod, dat, sigma)[0]  + tf.reduce_sum(-0.5*tf.log(2*np.pi*sig**2))
    #pp = np.argwhere((a1==-np.inf))
    #a1[pp] = 0
    return L


def chi2(mod, dat, sigma):
    sig = tf.cast(tf.diag_part(sigma), tf.float32) #np.diagonal(sigma)
    print(sig)
    AA = tf.reduce_sum(tf.square((dat - mod)/sig))
    BB = tf.reduce_sum((dat - mod)/tf.square(sig))
    CC = tf.reduce_sum(1/tf.square(sig))
    chi = AA - (BB**2)/CC
    return chi, BB/CC


def prior(theta, ndim): # log(pi)
    ct = 1
    r = np.eye(ndim)*ct
    print(r)
    print(theta)
    p = tf.log(tf.distributions.Normal(loc=theta, scale=r))
    return p


def EHubble(theta, z): # parametro de hubble
    om0 = theta[0]
    ol = theta[1]
    w = theta[2]
    arg = om0*(1 + z)**3 + (1 - om0 - ol)*(1 + z)**2 + ol*(1 + z)**(3*(1 + w))
    EE = tf.sqrt(arg)
    return EE, arg


def modelo(theta, z, x):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    om0 = theta[0]
    ol = theta[1]
    omega_k = 1 - om0 - ol
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300)) #sci.cumtrapz(1/(E + 1e-300), z, initial=0) + z[0]*((1/(E + 1e-300))[0] + 1)/2 # estabilidad numerica
    o_k_s = tf.sqrt(abs(omega_k))    
    omegak = 1 - x[0] - x[1]
    if omegak==0:
        dl = (1 + z)*I
    elif omegak>0:
        dl = (1 + z)*tf.sinh(o_k_s*I)/(o_k_s + 1e-300)
    else:
        dl = (1 + z)*tf.sin(o_k_s*I)/(o_k_s + 1e-300)
    dist = 5*tf.log(dl + 1e-300)/np.log(10)
    return dist


def revisa(theta, z):
    arg = EHubble(theta, z)[1]
    bol = np.sum(arg<0)
    print(bol)
    if bol>0:
        a = 0 # raiz imaginaria 
    else:
        a = 1 # raiz real
    return a


def revisa1(X):
    x = X[0]
    y = X[1]
    z = X[2]
    xlim = np.array([0, 1])
    ylim = xlim
    zlim = np.array([-np.inf, 1/3])
    if xlim[0]<x<xlim[1] and ylim[0]<y<ylim[1] and zlim[0]<z<zlim[1]:
        return 1
    else:
        return 0


def potencial(dat, sigma, theta, z, ndim):
    mod = modelo(theta, z)
    u = - likelihood(mod, dat, sigma) #- prior(theta, ndim) 
    return u


def cinetica(p, m):
    k = np.sum(p**2/2/m)
    return k


def hamiltoniano(p, dat, sigma, theta, z, m=1):
    h = cinetica(p, m) + potencial(dat, sigma, theta, z)
    return h


def gradiente(dw, theta, z, dat, sigma):
    tf = theta + dw
    tb = theta - dw
    grad = (potencial(dat, sigma, tf, z) - potencial(dat, sigma, tb, z))/(2*dw)
    return grad


def delta(x1, x2):
    return np.sum((x1 - x2)**2)

def lam(x1, x2, ac, par1):
    x = par1**2/delta(x1, x2)/ac
    return x - 1/x

def loss(x1, x2, y1, y2, ac1, ac2, par1, par2):
    return lam(x1, x2, ac1, par1) + par2*lam(y1, y2, ac2, par1)


In [96]:
# saved things directory
direc = '/home/mauricio/Documents/Uni/Intro_2/' + 'gal.txt'

# Carga de datos
redshift = np.genfromtxt('gal.txt', usecols=(1))
mu_obs = np.genfromtxt('gal.txt', usecols=(2)) # m - M
cov = np.genfromtxt('gal.txt', usecols=(3))

p = np.argsort(redshift)
redshift = redshift[p]
mu_obs = mu_obs[p]
cov = cov[p]
cov = np.diag(cov)

In [114]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=(3))
x = np.array([0.5, 0.4, -0.5])
x1 = tf.constant(x)
#mod = 

In [115]:
init = tf.global_variables_initializer()
#potencial(dat, sigma, theta_ten, theta, z)
with tf.Session() as sess:
    sess.run(init)
    x2 = x1.eval()
    print(sess.run(modelo(X, redshift, x2), feed_dict={X: x}))

[-9.1244469e+00 -9.1244469e+00 -9.1244469e+00 -9.1204767e+00
 -9.1098051e+00 -9.1001959e+00 -9.0952616e+00 -9.0952616e+00
 -9.0808134e+00 -9.0612726e+00 -8.9822683e+00 -8.9822683e+00
 -8.9413452e+00 -8.9385099e+00 -8.9351854e+00 -8.9211645e+00
 -8.9066133e+00 -8.8839808e+00 -8.8822699e+00 -8.8498783e+00
 -8.8264074e+00 -8.8194904e+00 -8.8101749e+00 -8.8101749e+00
 -8.7716770e+00 -8.7312260e+00 -8.6845589e+00 -8.6387634e+00
 -8.6153269e+00 -8.5806351e+00 -8.5806351e+00 -8.5733032e+00
 -8.5464773e+00 -8.5353203e+00 -8.4498281e+00 -8.4363232e+00
 -8.4043140e+00 -8.3937454e+00 -8.3727598e+00 -8.3623419e+00
 -8.3313808e+00 -8.3015547e+00 -8.2907610e+00 -8.2827244e+00
 -8.2707262e+00 -8.2707262e+00 -8.2410145e+00 -8.1923685e+00
 -8.1923685e+00 -8.1855297e+00 -8.1827650e+00 -8.1629219e+00
 -8.1542015e+00 -8.1542015e+00 -8.1319885e+00 -8.0981617e+00
 -8.0932789e+00 -8.0889568e+00 -8.0720024e+00 -8.0706635e+00
 -8.0603008e+00 -8.0412531e+00 -8.0255766e+00 -8.0166683e+00
 -8.0077963e+00 -7.99015